### Step 1 - Start Spark Session and Include additional configurations and common functions

In [1]:
%run "../includes/configurations"

In [2]:
%run "../includes/common_functions"

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Initialize a Spark session
spark = SparkSession.builder.appName("ConstructorsIngestion").getOrCreate()

23/12/29 11:18:24 WARN Utils: Your hostname, falcao-sys resolves to a loopback address: 127.0.1.1; using 192.168.11.185 instead (on interface wlx7898e8c12476)
23/12/29 11:18:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/29 11:18:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Step 2 - Read the JSON file sing the Spark Dataframe Reader

In [4]:
#constructors_schema = "constructorId INT, constructorRef STRING, name STRING, nationality STRING, url STRING"

In [5]:
constructors_schema = StructType(fields=[
    StructField("constructorId", IntegerType(), False),
    StructField("constructorRef", StringType(), True),
    StructField("name", StringType(), True),
    StructField("nationality", StringType(), True),
    StructField("url", StringType(), True)
])

In [6]:
constructor_df = spark.read.schema(constructors_schema).json("../data/constructors.json")

### Step 3 - Rename and drop columns, and add new columns

In [7]:
constructor_column_mapping = {
    "constructorId": "constructor_id",
    "constructorRef": "constructor_ref"
}

constructor_df = rename_columns(constructor_df, constructor_column_mapping)


In [8]:
columns_to_drop = ['url']

constructor_df = drop_columns(constructor_df, columns_to_drop)

In [9]:
constructor_df = add_ingestion_date(constructor_df)

### Step 4 - Write output to a parquet file

In [10]:
constructor_df.write.mode("overwrite").parquet("../processed_data/constructors")